# The latest RDS data

In [ ]:
library(Signac)
library(Seurat)
library(EnsDb.Hsapiens.v86)
library(BSgenome.Hsapiens.UCSC.hg38)
library(JASPAR2020)
library(TFBSTools)

set.seed(1234)

library(future)
plan("multicore", workers = 6)
plan()
options(future.globals.maxSize = 50 * 1024 ^ 3) # for 50 Gb RAM

In [ ]:
hm.integrated = readRDS("/mnt/data2/Project_outputs/Project2021_22/Gut_immune_surveillance/rds/combined_ATAC_epi_cluster.rds")
hm.integrated

# 处理.bed格式的fragments文件

https://github.com/timoast/signac/discussions/908

仅使用fragments作为Signac的输入。

**Server**: for-basic-calculations **8vCPUs | 64GiB**

**Environment**: **Python** [conda env:singlecell] [conda env: MACS] & **Shell**

**Data**: Ileum and Colon ATAC .bed /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/bed

**Time**: 2022.01.03-2022.01.05

**Researcher**: Wang Yue

**(1) 对原使用gzip压缩的.bed文件进行解压并sort，然后重新使用bgzip压缩**

使用bgzip对.tsv文件进行二值化压缩 to .tsv.gz，注意，不能使用gzip，否则无法进行索引操作（生成.tbi文件）。以其中一个为例：

Source（https://github.com/timoast/signac/issues/138）

In [ ]:
! cd /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/bed
! gzip -d -k Ileum_JF1O2.bed.gz
! bedtools sort -i Ileum_JF1O2.bed > Ileum_JF1O2_sort.bed
! bgzip -@ 7 Ileum_JF1O2_sort.bed

In [ ]:
# 使用tabix检查sort和bgzip处理后的bed.gz文件是否正确
! tabix -p bed Ileum_JF1O2_sort.bed.gz 

**(2) 将.bed格式的fragments文件处理为符合MACS3和Signac读取格式的fragments.tsv.gz文件，以及.tbi文件**

Convert .bed(6列) to .tsv(5列，Signac格式)

In [ ]:
file_in_path = '/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/bed/sort/'

file_out_path = '/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/'

In [ ]:
import os

def fine_file_name(file_in_path):
    for root, dirs, files in os.walk(file_in_path):
        print('The dataset has '+str(len(files))+' samples.')
        print('sub_dirs:', files)  # 当前路径下所有子目录
        file_name_list = files
        return file_name_list
    
file_name_list = fine_file_name(file_in_path)

In [ ]:
import pandas as pd
# 取dataframe文件的前5列，存储为TSV格式
for i in file_name_list:
    print('file '+i+' is being processed.')
    df = pd.read_csv(file_in_path+i, sep='\t', comment='t', header=None)
    df = df.loc[:,[0,1,2,3,4]]
    j = i.split('.')[0]
    print('file '+j+'.tsv'+' is being saved.')
    df.to_csv(file_out_path+j+'.tsv', sep='\t', header=None, index=False)

In [ ]:
# 将.tsv文件使用bgzip压缩为tsv.gz文件，并使用tabix命令生成.tbi文件
! cd /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/
! bgzip -@ 7 Ileum_JF1O2_sort.tsv
! tabix -p bed Ileum_JF1O2_sort.tsv.gz 

**(3) 使用MACS3处理fragments文件，生成类似10X CellRanger-ATAC输出的.bed文件和用于转换矩阵的.narrowPeak文件**

Ileum

In [ ]:
! conda activate MACS
! macs3 callpeak -t /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Ileum_JF1O2_sort.tsv.gz -g 2.7e+09 -f BED --nomodel --extsize 200 --shift -100 -n Ileum_JF1O2 --outdir /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS

In [ ]:
! macs3 callpeak -t /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Ileum_ADA5F_sort.tsv.gz -g 2.7e+09 -f BED --nomodel --extsize 200 --shift -100 -n Ileum_ADA5F --outdir /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS

In [ ]:
! macs3 callpeak -t /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Ileum_A62GO_sort.tsv.gz -g 2.7e+09 -f BED --nomodel --extsize 200 --shift -100 -n Ileum_A62GO --outdir /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS

Colon_transverse

In [ ]:
! macs3 callpeak -t /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Colon_transverse_CSSDA_sort.tsv.gz -g 2.7e+09 -f BED --nomodel --extsize 200 --shift -100 -n Colon_transverse_CSSDA --outdir /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS

In [ ]:
! macs3 callpeak -t /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Colon_transverse_BZ2ZS_sort.tsv.gz -g 2.7e+09 -f BED --nomodel --extsize 200 --shift -100 -n Colon_transverse_BZ2ZS --outdir /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS

In [ ]:
! macs3 callpeak -t /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Colon_transverse_ACCQ1_sort.tsv.gz -g 2.7e+09 -f BED --nomodel --extsize 200 --shift -100 -n Colon_transverse_ACCQ1 --outdir /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS

In [ ]:
! macs3 callpeak -t /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Colon_transverse_A9VP4_sort.tsv.gz -g 2.7e+09 -f BED --nomodel --extsize 200 --shift -100 -n Colon_transverse_A9VP4 --outdir /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS

In [ ]:
! macs3 callpeak -t /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Colon_transverse_A9HOW_sort.tsv.gz -g 2.7e+09 -f BED --nomodel --extsize 200 --shift -100 -n Colon_transverse_A9HOW --outdir /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS

Colon_sigmoid

In [ ]:
! macs3 callpeak -t /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Colon_sigmoid_JF1O8_sort.tsv.gz -g 2.7e+09 -f BED --nomodel --extsize 200 --shift -100 -n Colon_sigmoid_JF1O8 --outdir /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS

In [ ]:
! macs3 callpeak -t /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Colon_sigmoid_AZPYO_sort.tsv.gz -g 2.7e+09 -f BED --nomodel --extsize 200 --shift -100 -n Colon_sigmoid_AZPYO --outdir /mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS

# Signac-Merge data

**Server**: for-basic-calculations **8vCPUs | 64GiB**

**Environment**: **R [conda env:atac]**

**Time**: 2022.01.05

**Researcher**: Wang Yue

In [ ]:
library(Signac)
library(Seurat)
library(EnsDb.Hsapiens.v86)
library(BSgenome.Hsapiens.UCSC.hg38)

set.seed(1234)

library(future)
plan("multicore", workers = 6)
plan()
options(future.globals.maxSize = 50 * 1024 ^ 3) # for 50 Gb RAM

In [ ]:
annotation <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v86)
seqlevelsStyle(annotation) <- "UCSC"

## Combine_peaks

In [ ]:
# Peaks
Peakpath_Ileum_JF1O2 = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS/Ileum_JF1O2_peaks.narrowPeak"
Peakpath_Ileum_A62GO = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS/Ileum_A62GO_peaks.narrowPeak"
Peakpath_Ileum_ADA5F = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS/Ileum_ADA5F_peaks.narrowPeak"

Peakpath_Colon_JF1O8 = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS/Colon_sigmoid_JF1O8_peaks.narrowPeak"
Peakpath_Colon_AZPYO = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS/Colon_sigmoid_AZPYO_peaks.narrowPeak"

Peakpath_Colon_BZ2ZS = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS/Colon_transverse_BZ2ZS_peaks.narrowPeak"
Peakpath_Colon_A9VP4 = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS/Colon_transverse_A9VP4_peaks.narrowPeak"
Peakpath_Colon_CSSDA = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS/Colon_transverse_CSSDA_peaks.narrowPeak"
Peakpath_Colon_ACCQ1 = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS/Colon_transverse_ACCQ1_peaks.narrowPeak"
Peakpath_Colon_A9HOW = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/MACS/Colon_transverse_A9HOW_peaks.narrowPeak"

In [ ]:
df_Ileum_JF1O2 <- read.table(file = Peakpath_Ileum_JF1O2, 
                 col.names = c("chr", "start", "end", "name", "score", "strand", "fold_change", "neg_log10pvalue_summit", 
                               "neg_log10qvalue_summit", "relative_summit_position") )

df_Ileum_A62GO <- read.table(file = Peakpath_Ileum_A62GO, 
                 col.names = c("chr", "start", "end", "name", "score", "strand", "fold_change", "neg_log10pvalue_summit", 
                               "neg_log10qvalue_summit", "relative_summit_position") )

df_Ileum_ADA5F <- read.table(file = Peakpath_Ileum_ADA5F, 
                 col.names = c("chr", "start", "end", "name", "score", "strand", "fold_change", "neg_log10pvalue_summit", 
                               "neg_log10qvalue_summit", "relative_summit_position") )

df_Colon_JF1O8 <- read.table(file = Peakpath_Colon_JF1O8, 
                 col.names = c("chr", "start", "end", "name", "score", "strand", "fold_change", "neg_log10pvalue_summit", 
                               "neg_log10qvalue_summit", "relative_summit_position") )

df_Colon_AZPYO <- read.table(file = Peakpath_Colon_AZPYO, 
                 col.names = c("chr", "start", "end", "name", "score", "strand", "fold_change", "neg_log10pvalue_summit", 
                               "neg_log10qvalue_summit", "relative_summit_position") )

df_Colon_BZ2ZS <- read.table(file = Peakpath_Colon_BZ2ZS, 
                 col.names = c("chr", "start", "end", "name", "score", "strand", "fold_change", "neg_log10pvalue_summit", 
                               "neg_log10qvalue_summit", "relative_summit_position") )

df_Colon_A9VP4 <- read.table(file = Peakpath_Colon_A9VP4, 
                 col.names = c("chr", "start", "end", "name", "score", "strand", "fold_change", "neg_log10pvalue_summit", 
                               "neg_log10qvalue_summit", "relative_summit_position") )

df_Colon_CSSDA <- read.table(file = Peakpath_Colon_CSSDA, 
                 col.names = c("chr", "start", "end", "name", "score", "strand", "fold_change", "neg_log10pvalue_summit", 
                               "neg_log10qvalue_summit", "relative_summit_position") )

df_Colon_ACCQ1 <- read.table(file = Peakpath_Colon_ACCQ1, 
                 col.names = c("chr", "start", "end", "name", "score", "strand", "fold_change", "neg_log10pvalue_summit", 
                               "neg_log10qvalue_summit", "relative_summit_position") )

df_Colon_A9HOW <- read.table(file = Peakpath_Colon_A9HOW, 
                 col.names = c("chr", "start", "end", "name", "score", "strand", "fold_change", "neg_log10pvalue_summit", 
                               "neg_log10qvalue_summit", "relative_summit_position") )

In [ ]:
peaks_Ileum_JF1O2 <- makeGRangesFromDataFrame(df = df_Ileum_JF1O2, keep.extra.columns = TRUE)
peaks_Ileum_A62GO <- makeGRangesFromDataFrame(df = df_Ileum_A62GO, keep.extra.columns = TRUE)
peaks_Ileum_ADA5F <- makeGRangesFromDataFrame(df = df_Ileum_ADA5F, keep.extra.columns = TRUE)
peaks_Colon_JF1O8 <- makeGRangesFromDataFrame(df = df_Colon_JF1O8, keep.extra.columns = TRUE)
peaks_Colon_AZPYO <- makeGRangesFromDataFrame(df = df_Colon_AZPYO, keep.extra.columns = TRUE)
peaks_Colon_BZ2ZS <- makeGRangesFromDataFrame(df = df_Colon_BZ2ZS, keep.extra.columns = TRUE)
peaks_Colon_A9VP4 <- makeGRangesFromDataFrame(df = df_Colon_A9VP4, keep.extra.columns = TRUE)
peaks_Colon_CSSDA <- makeGRangesFromDataFrame(df = df_Colon_CSSDA, keep.extra.columns = TRUE)
peaks_Colon_ACCQ1 <- makeGRangesFromDataFrame(df = df_Colon_ACCQ1, keep.extra.columns = TRUE)
peaks_Colon_A9HOW <- makeGRangesFromDataFrame(df = df_Colon_A9HOW, keep.extra.columns = TRUE)

In [ ]:
# Create a unified set of peaks to quantify in each dataset
combined.peaks <- reduce(x = c(peaks_Ileum_JF1O2, peaks_Ileum_A62GO, peaks_Ileum_ADA5F,
                               peaks_Colon_JF1O8,peaks_Colon_AZPYO,peaks_Colon_BZ2ZS,
                               peaks_Colon_A9VP4,peaks_Colon_CSSDA,peaks_Colon_ACCQ1,
                               peaks_Colon_A9HOW))

# Filter out bad peaks based on length
peakwidths <- width(combined.peaks)
combined.peaks <- combined.peaks[peakwidths  < 10000 & peakwidths > 20]

# remove peaks on nonstandard chromosomes and in genomic blacklist regions
combined.peaks <- keepStandardChromosomes(combined.peaks, pruning.mode = "coarse")
combined.peaks <- subsetByOverlaps(x = combined.peaks, ranges = blacklist_hg38_unified, invert = TRUE)
combined.peaks

## create seurat objects

### Ileum_JF1O2

In [ ]:
fragpath = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Ileum_JF1O2_sort.tsv.gz"

In [ ]:
frags_object <- CreateFragmentObject(path = fragpath)

# quantify counts in each peak
macs2_counts <- FeatureMatrix(
  fragments = frags_object,
  features = combined.peaks
)

# create a new assay using the MACS2 peak set and add it to the Seurat object
chrom_assay <- CreateChromatinAssay(
  counts = macs2_counts,
  fragments = fragpath,
  sep = c(":", "-"),
  genome = 'hg38',
  annotation = annotation,
  #min.cells = 10,
  min.features = 200
)

Ileum_JF102 <- CreateSeuratObject(
  counts = chrom_assay,
  assay = "peaks"
)

Ileum_JF102

### Ileum_A62GO

In [ ]:
fragpath = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Ileum_A62GO_sort.tsv.gz"

In [ ]:
# create fragment objects
frags_object <- CreateFragmentObject(path = fragpath)

# quantify counts in each peak
macs2_counts <- FeatureMatrix(
  fragments = frags_object,
  features = combined.peaks
)

# create a new assay using the MACS2 peak set and add it to the Seurat object
chrom_assay <- CreateChromatinAssay(
  counts = macs2_counts,
  fragments = fragpath,
  sep = c(":", "-"),
  genome = 'hg38',
  annotation = annotation,
  #min.cells = 10,
  min.features = 200
)

Ileum_A62GO <- CreateSeuratObject(
  counts = chrom_assay,
  assay = "peaks"
)

Ileum_A62GO

### Ileum_ADA5F

In [ ]:
fragpath = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Ileum_ADA5F_sort.tsv.gz"

In [ ]:
# create fragment objects
frags_object <- CreateFragmentObject(path = fragpath)

# quantify counts in each peak
macs2_counts <- FeatureMatrix(
  fragments = frags_object,
  features = combined.peaks
)

# create a new assay using the MACS2 peak set and add it to the Seurat object
chrom_assay <- CreateChromatinAssay(
  counts = macs2_counts,
  fragments = fragpath,
  sep = c(":", "-"),
  genome = 'hg38',
  annotation = annotation,
  #min.cells = 10,
  min.features = 200
)

Ileum_ADA5F <- CreateSeuratObject(
  counts = chrom_assay,
  assay = "peaks"
)

Ileum_ADA5F

### Colon_sigmoid_JF1O8

In [ ]:
fragpath = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Colon_sigmoid_JF1O8_sort.tsv.gz"

In [ ]:
# create fragment objects
frags_object <- CreateFragmentObject(path = fragpath)

# quantify counts in each peak
macs2_counts <- FeatureMatrix(
  fragments = frags_object,
  features = combined.peaks
)

# create a new assay using the MACS2 peak set and add it to the Seurat object
chrom_assay <- CreateChromatinAssay(
  counts = macs2_counts,
  fragments = fragpath,
  sep = c(":", "-"),
  genome = 'hg38',
  annotation = annotation,
  #min.cells = 10,
  min.features = 200
)

Colon_sigmoid_JF1O8 <- CreateSeuratObject(
  counts = chrom_assay,
  assay = "peaks"
)

Colon_sigmoid_JF1O8

### Colon_sigmoid_AZPYO

In [ ]:
fragpath = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Colon_sigmoid_AZPYO_sort.tsv.gz"

In [ ]:
# create fragment objects
frags_object <- CreateFragmentObject(path = fragpath)

# quantify counts in each peak
macs2_counts <- FeatureMatrix(
  fragments = frags_object,
  features = combined.peaks
)

# create a new assay using the MACS2 peak set and add it to the Seurat object
chrom_assay <- CreateChromatinAssay(
  counts = macs2_counts,
  fragments = fragpath,
  sep = c(":", "-"),
  genome = 'hg38',
  annotation = annotation,
  #min.cells = 10,
  min.features = 200
)

Colon_sigmoid_AZPYO <- CreateSeuratObject(
  counts = chrom_assay,
  assay = "peaks"
)

Colon_sigmoid_AZPYO

### Colon_transverse_BZ2ZS

In [ ]:
fragpath = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Colon_transverse_BZ2ZS_sort.tsv.gz"

In [ ]:
# create fragment objects
frags_object <- CreateFragmentObject(path = fragpath)

# quantify counts in each peak
macs2_counts <- FeatureMatrix(
  fragments = frags_object,
  features = combined.peaks
)

# create a new assay using the MACS2 peak set and add it to the Seurat object
chrom_assay <- CreateChromatinAssay(
  counts = macs2_counts,
  fragments = fragpath,
  sep = c(":", "-"),
  genome = 'hg38',
  annotation = annotation,
  #min.cells = 10,
  min.features = 200
)

Colon_transverse_BZ2ZS <- CreateSeuratObject(
  counts = chrom_assay,
  assay = "peaks"
)

Colon_transverse_BZ2ZS

### Colon_transverse_A9VP4

In [ ]:
fragpath = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Colon_transverse_A9VP4_sort.tsv.gz"

In [ ]:
# create fragment objects
frags_object <- CreateFragmentObject(path = fragpath)

# quantify counts in each peak
macs2_counts <- FeatureMatrix(
  fragments = frags_object,
  features = combined.peaks
)

# create a new assay using the MACS2 peak set and add it to the Seurat object
chrom_assay <- CreateChromatinAssay(
  counts = macs2_counts,
  fragments = fragpath,
  sep = c(":", "-"),
  genome = 'hg38',
  annotation = annotation,
  #min.cells = 10,
  min.features = 200
)

Colon_transverse_A9VP4 <- CreateSeuratObject(
  counts = chrom_assay,
  assay = "peaks"
)

Colon_transverse_A9VP4

### Colon_transverse_CSSDA

In [ ]:
fragpath = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Colon_transverse_CSSDA_sort.tsv.gz"

In [ ]:
# create fragment objects
frags_object <- CreateFragmentObject(path = fragpath)

# quantify counts in each peak
macs2_counts <- FeatureMatrix(
  fragments = frags_object,
  features = combined.peaks
)

# create a new assay using the MACS2 peak set and add it to the Seurat object
chrom_assay <- CreateChromatinAssay(
  counts = macs2_counts,
  fragments = fragpath,
  sep = c(":", "-"),
  genome = 'hg38',
  annotation = annotation,
  #min.cells = 10,
  min.features = 200
)

Colon_transverse_CSSDA <- CreateSeuratObject(
  counts = chrom_assay,
  assay = "peaks"
)

Colon_transverse_CSSDA

### Colon_transverse_ACCQ1

In [ ]:
fragpath = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Colon_transverse_ACCQ1_sort.tsv.gz"

In [ ]:
# create fragment objects
frags_object <- CreateFragmentObject(path = fragpath)

# quantify counts in each peak
macs2_counts <- FeatureMatrix(
  fragments = frags_object,
  features = combined.peaks
)

# create a new assay using the MACS2 peak set and add it to the Seurat object
chrom_assay <- CreateChromatinAssay(
  counts = macs2_counts,
  fragments = fragpath,
  sep = c(":", "-"),
  genome = 'hg38',
  annotation = annotation,
  #min.cells = 10,
  min.features = 200
)

Colon_transverse_ACCQ1 <- CreateSeuratObject(
  counts = chrom_assay,
  assay = "peaks"
)

Colon_transverse_ACCQ1

### Colon_transverse_A9HOW

In [ ]:
fragpath = "/mnt/data2/Datasets/ATAC_data/Cell2021_human_adult_multiple_organs_scATAC/TSV/Colon_transverse_A9HOW_sort.tsv.gz"

In [ ]:
# create fragment objects
frags_object <- CreateFragmentObject(path = fragpath)

# quantify counts in each peak
macs2_counts <- FeatureMatrix(
  fragments = frags_object,
  features = combined.peaks
)

# create a new assay using the MACS2 peak set and add it to the Seurat object
chrom_assay <- CreateChromatinAssay(
  counts = macs2_counts,
  fragments = fragpath,
  sep = c(":", "-"),
  genome = 'hg38',
  annotation = annotation,
  #min.cells = 10,
  min.features = 200
)

Colon_transverse_A9HOW <- CreateSeuratObject(
  counts = chrom_assay,
  assay = "peaks"
)

Colon_transverse_A9HOW

# Merge objects

In [ ]:
# add information to identify dataset of origin
Ileum_JF102$dataset <- 'Ileum_JF102'
Ileum_A62GO$dataset <- 'Ileum_A62GO'
Ileum_ADA5F$dataset <- 'Ileum_ADA5F'
Colon_sigmoid_JF1O8$dataset <- 'Colon_sigmoid_JF1O8'
Colon_sigmoid_AZPYO$dataset <- 'Colon_sigmoid_AZPYO'
Colon_transverse_BZ2ZS$dataset <- 'Colon_transverse_BZ2ZS'
Colon_transverse_A9VP4$dataset <- 'Colon_transverse_A9VP4'
Colon_transverse_CSSDA$dataset <- 'Colon_transverse_CSSDA'
Colon_transverse_ACCQ1$dataset <- 'Colon_transverse_ACCQ1'
Colon_transverse_A9HOW$dataset <- 'Colon_transverse_A9HOW'

In [ ]:
Ileum_JF102$organ <- 'Ileum'
Ileum_A62GO$organ <- 'Ileum'
Ileum_ADA5F$organ <- 'Ileum'
Colon_sigmoid_JF1O8$organ <- 'Colon'
Colon_sigmoid_AZPYO$organ <- 'Colon'
Colon_transverse_BZ2ZS$organ <- 'Colon'
Colon_transverse_A9VP4$organ <- 'Colon'
Colon_transverse_CSSDA$organ <- 'Colon'
Colon_transverse_ACCQ1$organ <- 'Colon'
Colon_transverse_A9HOW$organ <- 'Colon'

In [ ]:
# merge all datasets, adding a cell ID to make sure cell names are unique
combined <- merge(
  x = Ileum_JF102,
  y = list(Ileum_A62GO, Ileum_ADA5F, Colon_sigmoid_JF1O8, Colon_sigmoid_AZPYO,
           Colon_transverse_BZ2ZS,Colon_transverse_A9VP4,Colon_transverse_CSSDA,Colon_transverse_ACCQ1,Colon_transverse_A9HOW),
  add.cell.ids = c('JF102', 'A62GO', 'ADA5F','JF1O8','AZPYO','BZ2ZS','A9VP4','CSSDA','ACCQ1','A9HOW')
)
combined

In [ ]:
saveRDS(object = combined, file = "/mnt/data2/Project_outputs/Project2021_22/Gut_immune_surveillance/rds/combined_ATAC.rds")

# Computing QC Metrics

In [ ]:
combined <- readRDS("/mnt/data2/Project_outputs/Project2021_22/Gut_immune_surveillance/rds/combined_ATAC.rds")
combined

In [ ]:
head(combined@meta.data)

In [ ]:
# compute nucleosome signal score per cell
combined <- NucleosomeSignal(object = combined)

# compute TSS enrichment score per cell
combined <- TSSEnrichment(object = combined, fast = FALSE)

In [ ]:
combined

In [ ]:
combined$high.tss <- ifelse(combined$TSS.enrichment > 2, 'High', 'Low')
TSSPlot(combined, group.by = 'high.tss') + NoLegend()

In [ ]:
combined$nucleosome_group <- ifelse(combined$nucleosome_signal > 4, 'NS > 4', 'NS < 4')
FragmentHistogram(object = combined, group.by = 'nucleosome_group')

In [ ]:
DefaultAssay(combined) <- "peaks"

VlnPlot(
  object = combined,
  features = c("nCount_peaks",'TSS.enrichment','nucleosome_signal'),
  pt.size = 0, #不展示dot
  ncol = 3
)

In [ ]:
# filter out low quality cells (使用文献Cell2021原文的过滤标准，TSS.enrichment似乎定位>7？)
combined <- subset(
  x = combined,
  subset = nCount_peaks < 100000 &
    nCount_peaks > 1000 &
    nucleosome_signal < 2 &
    TSS.enrichment > 1
)
combined

In [ ]:
saveRDS(object = combined, file = "/mnt/data2/Project_outputs/Project2021_22/Gut_immune_surveillance/rds/combined_ATAC_filtered.rds")

# Normalization, Feature selection and Dimension reduction

In [ ]:
combined <- FindTopFeatures(combined, min.cutoff = "q5")
combined <- RunTFIDF(combined)
combined <- RunSVD(combined)

In [ ]:
DepthCor(combined)

In [ ]:
combined <- RunUMAP(object = combined, reduction = 'lsi', dims = 2:30)
combined <- FindNeighbors(object = combined, reduction = 'lsi', dims = 2:30)
combined <- FindClusters(object = combined, verbose = FALSE, algorithm = 3)
DimPlot(object = combined, label = TRUE) + NoLegend()

In [ ]:
saveRDS(object = combined, file = "/mnt/data2/Project_outputs/Project2021_22/Gut_immune_surveillance/rds/combined_ATAC_cluster.rds")

# Create a gene activity matrix

In [ ]:
gene.activities <- GeneActivity(combined)

In [ ]:
# add the gene activity matrix to the Seurat object as a new assay and normalize it
combined[['RNA']] <- CreateAssayObject(counts = gene.activities)
combined <- NormalizeData(
  object = combined,
  assay = 'RNA',
  normalization.method = 'LogNormalize',
  scale.factor = median(combined$nCount_RNA)
)

In [ ]:
DefaultAssay(combined) <- 'RNA'

FeaturePlot(
  object = combined,
  features = c('EPCAM'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 1
)

FeaturePlot(
  object = combined,
  features = c('PTPRC'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 1
)

FeaturePlot(
  object = combined,
  features = c('CD3D'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 1
)

FeaturePlot(
  object = combined,
  features = c('CD79A'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 1
)

FeaturePlot(
  object = combined,
  features = c('CD14'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 1
)

FeaturePlot(
  object = combined,
  features = c('IGFBP7'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 1
)

In [ ]:
DimPlot(combined, group.by = 'organ', pt.size = 0.1)

In [ ]:
# 未进行批次校正
DimPlot(combined, group.by = 'dataset', pt.size = 0.1)

In [ ]:
FeaturePlot(
  object = combined,
  features = c('DEFA5'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 1
)

FeaturePlot(
  object = combined,
  features = c('DEFA6'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 1
)

FeaturePlot(
  object = combined,
  features = c('REG3A'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 1
)

FeaturePlot(
  object = combined,
  features = c('REG3G'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 1
)

In [ ]:
FeaturePlot(
  object = combined,
  features = c('LGR5'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 1
)

FeaturePlot(
  object = combined,
  features = c('FABP6'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 1
)

FeaturePlot(
  object = combined,
  features = c('AQP8'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 1
)

FeaturePlot(
  object = combined,
  features = c('MUC2'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 1
)

FeaturePlot(
  object = combined,
  features = c('CHGA'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 1
)

In [ ]:
combined

In [ ]:
saveRDS(object = combined, file = "/mnt/data2/Project_outputs/Project2021_22/Gut_immune_surveillance/rds/combined_ATAC_cluster.rds")

# Sort epithlial cells

In [ ]:
combined@assays

In [ ]:
head(combined@meta.data)

In [ ]:
DimPlot(object = combined, label = TRUE) + NoLegend()

In [ ]:
combined_epi <- combined[,combined@meta.data$seurat_clusters %in% c(5,7,14,20,22,6,12,1,4,18)]
combined_epi

In [ ]:
DimPlot(object = combined_epi, label = TRUE) + NoLegend()

In [ ]:
saveRDS(object = combined_epi, file = "/mnt/data2/Project_outputs/Project2021_22/Gut_immune_surveillance/rds/combined_ATAC_epi.rds")

# Batch removed with Harmony

In [ ]:
library(harmony)

In [ ]:
DefaultAssay(combined_epi) <- "peaks"

# 使用RunHarmony函数进行数据整合
hm.integrated <- RunHarmony(
  object = combined_epi,
  group.by.vars = 'dataset',
  reduction = 'lsi',
  assay.use = 'peaks',
  project.dim = FALSE
)

In [ ]:
# re-compute the UMAP using corrected LSI embeddings
# 数据降维可视化
hm.integrated <- RunUMAP(hm.integrated, dims = 2:30, reduction = 'harmony')

In [ ]:
DimPlot(hm.integrated, group.by = 'dataset', pt.size = 0.1) + ggplot2::ggtitle("Harmony integration")
DimPlot(hm.integrated, group.by = 'organ', pt.size = 0.1) + ggplot2::ggtitle("Harmony integration")

In [ ]:
hm.integrated

In [ ]:
hm.integrated <- FindNeighbors(object = hm.integrated, dims = 2:30, reduction = 'harmony')
hm.integrated

In [ ]:
hm.integrated <- FindClusters(object = hm.integrated, resolution = 0.5)

In [ ]:
DimPlot(object = hm.integrated, label = TRUE) + NoLegend()

In [ ]:
FeaturePlot(
  object = hm.integrated,
  features = c('CLCA1'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 1
)

FeaturePlot(
  object = hm.integrated,
  features = c('BEST2'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 1
)

FeaturePlot(
  object = hm.integrated,
  features = c('CFTR'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 1
)

FeaturePlot(
  object = hm.integrated,
  features = c('FCGBP'),
  pt.size = 0.1,
  max.cutoff = 'q95',
  ncol = 1
)

In [ ]:
DefaultAssay(hm.integrated) <- 'RNA'
FeaturePlot(
  object = hm.integrated,
  features = c('LGR5'),
  pt.size = 0.5,
  max.cutoff = 'q95',
  ncol = 1
)

In [ ]:
FeaturePlot(
  object = hm.integrated,
  features = c('TCF4'),
  pt.size = 0.5,
  max.cutoff = 'q95',
  ncol = 1
)

# Find differentially accessible peaks between clusters

In [ ]:
# change back to working with gene activities instead of paeks
DefaultAssay(hm.integrated) <- 'RNA'

da_peaks <- FindMarkers(
  object = hm.integrated,
  ident.1 = c("1"),
  ident.2 = c("0","2","3","4","5"),
  min.pct = 0.05,
  test.use = 'LR',
)

# cluster 1 is BEST2+ goblet cells
head(da_peaks)

In [ ]:
da_peaks

In [ ]:
# change back to working with peaks instead of gene activities
DefaultAssay(hm.integrated) <- 'peaks'

da_peaks <- FindMarkers(
  object = hm.integrated,
  ident.1 = c("1"),
  ident.2 = c("0","2","3","4","5"),
  min.pct = 0.05,
  test.use = 'LR',
)

# cluster 1 is BEST2+ goblet cells
head(da_peaks)

In [ ]:
fc <- FoldChange(hm.integrated, ident.1 = c("1"), ident.2 = c("0","2","3","4","5"))
head(fc)

In [ ]:
open_c1 <- rownames(da_peaks[da_peaks$avg_log2FC > 0.5, ])
open_others <- rownames(da_peaks[da_peaks$avg_log2FC < -0.5, ])

In [ ]:
open_c1

In [ ]:
open_others

In [ ]:
closest_genes_c1 <- ClosestFeature(hm.integrated, regions = open_c1)

In [ ]:
closest_genes_c1

In [ ]:
# change back to working with peaks instead of gene activities
DefaultAssay(hm.integrated) <- 'peaks'

da_peaks <- FindMarkers(
  object = hm.integrated,
  ident.1 = c("6"),
  ident.2 = c("0","1","2","3","4","5"),
  min.pct = 0.05,
  test.use = 'LR',
)

# cluster 1 is BEST2+ goblet cells
head(da_peaks)

In [ ]:
open_c6 <- rownames(da_peaks[da_peaks$avg_log2FC > 0.3, ])
closest_genes_c6 <- ClosestFeature(hm.integrated, regions = open_c6)

In [ ]:
closest_genes_c6

# Plotting genomic regions

In [ ]:
# DEFA5, WFDC2
'chr8-7055304-7056739' 'chr20-45469753-45481532'

In [ ]:
options(repr.plot.height = 7, repr.plot.width = 10)

In [ ]:
# set plotting order
# levels(pbmc) <- c("CD4 Naive","CD4 Memory","CD8 Naive","CD8 Effector","DN T","NK CD56bright","NK CD56Dim","pre-B",'pro-B',"pDC","DC","CD14 Mono",'CD16 Mono')
# DEFA5

CoveragePlot(
  object = hm.integrated,
#  group.by = 'organ',
  region = 'chr8-7055304-7056739',
  extend.upstream = 5000,
  extend.downstream = 2000
)

CoveragePlot(
  object = hm.integrated,
  group.by = 'organ',
  region = 'chr8-7055304-7056739',
  extend.upstream = 5000,
  extend.downstream = 2000
)

In [ ]:
# 提取Paneth cells (c(6))
Paneth <- hm.integrated[,hm.integrated@meta.data$seurat_clusters %in% c(6)]
Paneth

In [ ]:
options(repr.plot.height = 7, repr.plot.width = 15)

ranges.show <- StringToGRanges("chr8-7057234-7057248") # NR1H4

options(repr.plot.height = 9, repr.plot.width = 14)

CoveragePlot(
  object = Paneth,
  group.by = 'organ',
  region = 'chr8-7055304-7056739',
  extend.upstream =  0,
  extend.downstream = 2000,
  region.highlight = ranges.show
)

In [ ]:
options(repr.plot.height = 7, repr.plot.width = 15)

ranges.show <- StringToGRanges("chr8-7057323-7057337") # VDRchr20-45469620-45469630

options(repr.plot.height = 9, repr.plot.width = 14)

CoveragePlot(
  object = Paneth,
  group.by = 'organ',
  region = 'chr8-7055304-7056739',
  extend.upstream = 2000,
  extend.downstream = 2000,
  region.highlight = ranges.show
)

In [ ]:
options(repr.plot.height = 7, repr.plot.width = 15)

ranges.show <- StringToGRanges("chr8-7056889-7056899") # TCF7 chr8:7056889-7056899

options(repr.plot.height = 9, repr.plot.width = 14)

CoveragePlot(
  object = Paneth,
  group.by = 'organ',
  region = 'chr8-7055304-7056739',
  extend.upstream = 2000,
  extend.downstream = 2000,
  region.highlight = ranges.show
)

In [ ]:
options(repr.plot.height = 7, repr.plot.width = 15)

ranges.show <- StringToGRanges("chr20-45469620-45469630") # NR1H4

options(repr.plot.height = 9, repr.plot.width = 14)

CoveragePlot(
  object = hm.integrated,
  group.by = 'organ',
  region = 'chr20-45469753-45481532',
  extend.upstream = 5000,
  extend.downstream = 2000,
  region.highlight = ranges.show
)

In [ ]:
CoveragePlot(
  object = Paneth,
  group.by = 'organ',
  region = 'chr8-6924697-6926076',
  extend.upstream = 5000,
  extend.downstream = 2000,
)

In [ ]:
CoveragePlot(
  object = Paneth,
  group.by = 'organ',
  region = 'chr8-6924697-7056739',
  extend.upstream = 5000,
  extend.downstream = 2000,
)

In [ ]:
CoveragePlot(
  object = Paneth,
  group.by = 'organ',
  region = 'chr2-79157003-79159753',
  extend.upstream = 5000,
  extend.downstream = 2000,
)

In [ ]:
CoveragePlot(
  object = Paneth,
  group.by = 'organ',
  region = 'chr2-79025664-79028505',
  extend.upstream = 5000,
  extend.downstream = 2000,
)

In [ ]:
# DEFA6: chr8-6924697-6926076

CoveragePlot(
  object = hm.integrated,
#  group.by = 'organ',
  region = 'chr8-6924697-6926076',
  extend.upstream = 2000,
  extend.downstream = 2000
)

CoveragePlot(
  object = hm.integrated,
  group.by = 'organ',
  region = 'chr8-6924697-6926076',
  extend.upstream = 2000,
  extend.downstream = 2000
)

In [ ]:
# WFDC2
CoveragePlot(
  object = hm.integrated,
  #group.by = 'organ',
  region = 'chr20-45469753-45481532',
  extend.upstream = 5000,
  extend.downstream = 2000
)

CoveragePlot(
  object = hm.integrated,
  group.by = 'organ',
  region = 'chr20-45469753-45481532',
  extend.upstream = 5000,
  extend.downstream = 2000
)

In [ ]:
# ADM: chr11-10305073-10307397
CoveragePlot(
  object = hm.integrated,
  #group.by = 'organ',
  region = 'chr11-10305073-10307397',
  extend.upstream = 5000,
  extend.downstream = 200
)

CoveragePlot(
  object = hm.integrated,
  group.by = 'organ',
  region = 'chr11-10305073-10307397',
  extend.upstream = 5000,
  extend.downstream = 200
)

In [ ]:
# DEFB1: chr8-6870592-6877936
CoveragePlot(
  object = hm.integrated,
  #group.by = 'organ',
  region = 'chr8-6870592-6877936',
  extend.upstream = 5000,
  extend.downstream = 2000
)

CoveragePlot(
  object = hm.integrated,
  group.by = 'organ',
  region = 'chr8-6870592-6877936',
  extend.upstream = 5000,
  extend.downstream = 2000
)

In [ ]:
CoveragePlot(
  object = hm.integrated,
  #group.by = 'organ',
  region = 'chr20-45174902-45176544',
  extend.upstream = 5000,
  extend.downstream = 2000
)

CoveragePlot(
  object = hm.integrated,
  group.by = 'organ',
  region = 'chr20-45174902-45176544',
  extend.upstream = 5000,
  extend.downstream = 2000
)

In [ ]:
CoveragePlot(
  object = hm.integrated,
  #group.by = 'organ',
  region = 'chr2-227805739-227817564',
  extend.upstream = 0,
  extend.downstream = 0
)

CoveragePlot(
  object = hm.integrated,
  group.by = 'organ',
  region = 'chr2-227805739-227817564',
  extend.upstream = 0,
  extend.downstream = 0
)

In [ ]:
CoveragePlot(
  object = hm.integrated,
  #group.by = 'organ',
  region = 'chr17-36064272-36072032',
  extend.upstream = 5000,
  extend.downstream = 0
)

CoveragePlot(
  object = hm.integrated,
  group.by = 'organ',
  region = 'chr17-36064272-36072032',
  extend.upstream = 5000,
  extend.downstream = 0
)

In [ ]:
saveRDS(object = hm.integrated, file = "/mnt/data2/Project_outputs/Project2021_22/Gut_immune_surveillance/rds/combined_ATAC_epi_cluster.rds")

# Motif analysis

In [ ]:
# 使用BiocManager install JASPAR2020,TFBSTools,motifmatchr并导入如下R包：
library(JASPAR2020)
library(TFBSTools)

In [ ]:
hm.integrated

In [ ]:
p1 <- DimPlot(hm.integrated, label = TRUE, pt.size = 0.1) + NoLegend()
p1

## All epithelial cells

**Adding motif information to the Seurat object**

To add the DNA sequence motif information required for motif analyses, we can run the AddMotifs() function:

In [ ]:
# Get a list of motif position frequency matrices from the JASPAR database
pfm <- getMatrixSet(
  x = JASPAR2020,
  opts = list(collection = "CORE", tax_group = 'vertebrates', all_versions = FALSE)
)

# add motif information
hm.integrated <- AddMotifs(
  object = hm.integrated,
  genome = BSgenome.Hsapiens.UCSC.hg38,
  pfm = pfm
)

To facilitate motif analysis in Signac, we have create the Motif class to store all the required information, including a list of position weight matrices (PWMs) or position frequency matrices (PFMs) and a motif occurrence matrix. Here, the AddMotifs() function construct a Motif object and adds it to our mouse brain dataset, along with other information such as the base composition of each peak. A motif object can be added to any Seurat assay using the SetAssayData() function.

为了便于Signac中的motif分析，我们创建了motif类来存储所有必需的信息，包括位置权重矩阵（PWM）或位置频率矩阵（PFM）列表以及motif发生矩阵。这里，addMotions（）函数构造一个Motif对象，并将其与其他信息（如每个峰值的基本组成）一起添加到我们的数据集中。可以使用SetAssayData（）函数将motif对象添加到任何Seurat分析中。

**Finding overrepresented motifs**

To identify potentially important cell-type-specific regulatory sequences, we can search for DNA motifs that are overrepresented in a set of peaks that are differentially accessible between cell types.

Here, we find differentially accessible peaks between Pvalb and Sst inhibitory interneurons. For sparse data (such as scATAC-seq), we find it is often necessary to lower the min.pct threshold in FindMarkers() from the default (0.1, which was designed for scRNA-seq data).

We then perform a hypergeometric test to test the probability of observing the motif at the given frequency by chance, comparing with a background set of peaks matched for GC content.

为了识别潜在重要的细胞类型特异性调控序列，我们可以搜索在一组峰中过度表达的DNA基序，这些峰在细胞类型之间差异可及。

在这里，我们发现Pvalb和Sst抑制中间神经元之间的差异可及峰。对于稀疏数据（如scATAC seq），我们发现通常需要将FindMarkers（）中的min.pct阈值从默认值（0.1，设计用于scRNA seq数据）降低。

然后，我们进行超几何测试，以测试在给定频率下偶然观察到基序的概率，并将其与GC含量匹配的背景峰集进行比较。

In [ ]:
da_peaks <- FindMarkers(
  object = hm.integrated,
  group.by = 'organ',
  ident.1 = 'Ileum',
  ident.2 = 'Colon',
  only.pos = TRUE,
  test.use = 'LR',
  min.pct = 0.05,
  latent.vars = 'nCount_peaks'
)

# get top differentially accessible peaks
top.da.peak <- rownames(da_peaks[da_peaks$p_val < 0.005, ])

In [ ]:
## Choosing a set of background peaks
# test enrichment
enriched.motifs <- FindMotifs(
  object = hm.integrated,
  features = top.da.peak
)

In [ ]:
enriched.motifs

In [ ]:
write.table(enriched.motifs,"/mnt/data2/Project_outputs/Project2021_22/Gut_immune_surveillance/outputs/epi_motifs.csv",row.names=TRUE,col.names=TRUE,sep=",")

In [ ]:
MotifPlot(
  object = hm.integrated,
  motifs = head(rownames(enriched.motifs))
)

We and others have previously shown that Mef-family motifs, particularly Mef2c, are enriched in Pvalb-specific peaks in scATAC-seq data (https://doi.org/10.1016/j.cell.2019.05.031; https://doi.org/10.1101/615179), and further shown that Mef2c is required for the development of Pvalb interneurons (https://www.nature.com/articles/nature25999). Here our results are consistent with these findings, and we observe a strong enrichment of Mef-family motifs in the top results from FindMotifs()

我们和其他人之前已经表明，Mef家族基序，特别是Mef2c，在scATAC-seq数据中的Pvalb特异性峰中富集，并进一步表明Mef2c是Pvalb中间神经元发育所必需的(https://www.nature.com/articles/nature25999).在这里，我们的结果与这些发现一致，并且我们观察到FindMotifs（）的顶级结果中Mef家族基序的强烈富集。

## Paneth cells

In [ ]:
head(hm.integrated@meta.data)

In [ ]:
Paneth <- hm.integrated[,hm.integrated@meta.data$seurat_clusters %in% c(6)]
Paneth

In [ ]:
DefaultAssay(Paneth) <- 'peaks'

da_peaks <- FindMarkers(
  object = Paneth,
  group.by = 'organ',
  ident.1 = 'Ileum',
  ident.2 = 'Colon',
  only.pos = TRUE,
  test.use = 'LR',
  min.pct = 0.05,
  latent.vars = 'nCount_peaks'
)

# get top differentially accessible peaks
top.da.peak <- rownames(da_peaks[da_peaks$p_val < 0.005, ])

In [ ]:
Paneth

In [ ]:
## Choosing a set of background peaks
# test enrichment
enriched.motifs <- FindMotifs(
  object = Paneth,
  features = top.da.peak
)

In [ ]:
enriched.motifs

# Computing motif activities

We can also compute a per-cell motif activity score by running chromVAR. This allows us to visualize motif activities per cell, and also provides an alternative method of identifying differentially-active motifs between cell types.
ChromVAR identifies motifs associated with variability in chromatin accessibility between cells. See the chromVAR paper for a complete description of the method.

我们还可以通过运行chromVAR计算每个细胞的基序活动分数。这使我们能够可视化每个细胞的基序活动，也提供了一种识别细胞类型之间差异活性基序的替代方法。
ChromVAR识别与细胞间染色质可及性变异相关的基序。有关该方法的完整说明，请参阅chromVAR论文。

In [ ]:
hm.integrated

In [ ]:
hm.integrated <- RunChromVAR(
  object = hm.integrated,
  genome = BSgenome.Hsapiens.UCSC.hg38
)

In [ ]:
hm.integrated

In [ ]:
DefaultAssay(hm.integrated) <- 'chromvar'

In [ ]:
# look at the activity of NR1H4
FeaturePlot(
  object = hm.integrated,
  split.by = 'organ',
  features = "MA0830.2",
  min.cutoff = 'q10',
  max.cutoff = 'q90',
  pt.size = 0.1
)

In [ ]:
# look at the activity of NR1H4
FeaturePlot(
  object = hm.integrated,
  split.by = 'organ',
  features = "MA1146.1",
  min.cutoff = 'q10',
  max.cutoff = 'q90',
  pt.size = 0.1
)

In [ ]:
# look at the activity of HNF4A
FeaturePlot(
  object = hm.integrated,
  split.by = 'organ',
  features = "MA1494.1",
  min.cutoff = 'q10',
  max.cutoff = 'q90',
  pt.size = 0.1
)

In [ ]:
# look at the activity of VDR
FeaturePlot(
  object = hm.integrated,
  split.by = 'organ',
  features = "MA0693.2",
  min.cutoff = 'q10',
  max.cutoff = 'q90',
  pt.size = 0.1
)

In [ ]:
# look at the activity of VDR
FeaturePlot(
  object = hm.integrated,
  split.by = 'organ',
  features = "MA0162.4",
  min.cutoff = 'q10',
  max.cutoff = 'q90',
  pt.size = 0.1
)

In [ ]:
saveRDS(object = hm.integrated, file = "/mnt/data2/Project_outputs/Project2021_22/Gut_immune_surveillance/rds/combined_ATAC_epi_cluster.rds")